#### Actualmente hay 741 municipios donde hubo incendio y solo tenemos 233 estaciones metorológicas. Por lo que si hacemos join de incencios con valores meteorológicos perderemos muchos datos sobre incendios. 
#### El objetivo es tratar de asignar la estacion meteorológica XEMA más cercana  a cada municicipio.

In [0]:
!pip install geopy tqdm

Requirement already satisfied: geopy in /databricks/python3/lib/python3.8/site-packages (2.3.0)
Requirement already satisfied: tqdm in /databricks/python3/lib/python3.8/site-packages (4.64.1)
Requirement already satisfied: geographiclib<3,>=1.52 in /databricks/python3/lib/python3.8/site-packages (from geopy) (2.0)
 WARNING: You are using pip version 21.0.1; however, version 22.3.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
from functools import reduce
import pyspark.sql.functions as F
from geopy.geocoders import Nominatim
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.types import StringType
import pandas as pd

In [0]:
#parquet
incendios_ocurridos_CATDF = spark.read.format("parquet") \
.load("/mnt/IncendiosForestalesCAT/raw/meteocat/incendios_forestales") \
.select(["codi_municipi","termemunic"]) \
.withColumnRenamed("termemunic","municipio") \
.withColumnRenamed("codi_municipi","codigo_municipio")

#parquet
alt_pend_riscDF = spark.read.format("parquet") \
.load("/mnt/IncendiosForestalesCAT/prep/alt_pend_risc") \
.select(["cod_idescat","municipio"]) \
.withColumnRenamed("cod_idescat","codigo_municipio")


In [0]:
incendios_ocurridos_CATDF.select(["municipio"]).distinct().count()

Out[4]: 741

In [0]:
alt_pend_riscDF.select(["municipio"]).distinct().count()

Out[5]: 946

In [0]:
# Obtener municipios que no tienen datos meteo
muni_incendios= incendios_ocurridos_CATDF.distinct()
muni_risc =alt_pend_riscDF.distinct()
missing_muni = muni_risc.alias("df1").join(muni_incendios.alias("df2"), F.col("df1.codigo_municipio")==F.col("df2.codigo_municipio"),"left")
missing_muni.filter(F.col("df2.municipio").isNull()).count()

Out[6]: 206

In [0]:
missing_muni=missing_muni.select("df1.*")
display(missing_muni)

codigo_municipio,municipio
89019,Rupit i Pruit
431711,Vila-seca
250241,Alt Àneu
82074,Sant Esteve de Palautordera
80790,"Estany, l'"
80272,"Cabanyes, les"
82270,Sant Martí Sarroca
170524,Siurana
430976,"Nou de Gaià, la"
81115,Malla


Solo tenemos 1 municipio sin datos de cartografia, riesgo incendio y 206 municipios que no tenemos en la tabla incendio. 
Por lo tanto se analizará calculará la distancia entre municipios de alt_pend_riscDF y las estaciones AEMET y Meteocat

In [0]:
estaciones_meteocatDF = spark.read.parquet("/mnt/IncendiosForestalesCAT/raw/meteocat/estaciones/")\
.filter(F.col("data_fi") >= "nan")\
.select(["codi_estacio", "latitud", "longitud", "nom_municipi"])\
.withColumnRenamed("codi_estacio","indicativo")\
.withColumnRenamed("nom_municipi","nombre")
display(estaciones_meteocatDF)

indicativo,latitud,longitud,nombre
X4,41.3839,2.16775,Barcelona
YG,42.51881,1.24244,Tírvia
VY,41.25095,1.29863,Nulles
YD,41.51135,0.85617,Les Borges Blanques
VD,41.68939,1.20381,Els Plans de Sió
WX,41.9178,0.88175,Camarasa
XV,41.48311,2.07956,Sant Cugat del Vallès
U1,42.30648,2.95481,Cabanes
CY,41.87813,2.17873,Muntanyola
D6,42.43515,3.16622,Portbou


In [0]:
@udf(returnType=StringType())
def convertToDec(coord_str):
    """
    Convertir coordenada con formato grados, minutos, segundos a decimales
    """
    direction = coord_str[-1]
    coord = (int(coord_str[:2]) +int(coord_str[2:4])/60 +float(coord_str[4:6])/3600)* (-1 if direction in ['W', 'S'] else 1)
    return (f"{coord}")

In [0]:
estaciones_aemetDF = spark.read.parquet("/mnt/IncendiosForestalesCAT/raw/aemet/estaciones/")\
.filter(F.col("provincia").isin(["BARCELONA", "TARRAGONA", "GIRONA", "LLEIDA"]))\
.select(["indicativo", "latitud", "longitud", "nombre"]) \
.withColumn("latitud", convertToDec(F.col("latitud"))) \
.withColumn("longitud", convertToDec(F.col("longitud")))
display(estaciones_aemetDF)

indicativo,latitud,longitud,nombre
0252D,41.587500000000006,2.54,ARENYS DE MAR
0076,41.29277777777778,2.0700000000000003,BARCELONA AEROPUERTO
0200E,41.41833333333333,2.1241666666666665,"BARCELONA, FABRA"
0201D,41.39055555555556,2.2,BARCELONA
0149X,41.72,1.840277777777778,MANRESA
0229I,41.52361111111111,2.1030555555555557,SABADELL AEROPUERTO
0255B,41.65083333333333,2.6969444444444446,SANTA SUSANNA
0367,41.91166666666666,2.763333333333333,GIRONA AEROPUERTO
0370B,41.980000000000004,2.8252777777777776,"GIRONA, ANTIC INSTITUT"
0372C,42.10444444444445,2.763611111111111,PORQUERES


In [0]:
estacionesDF= estaciones_meteocatDF.union(estaciones_aemetDF)
display(estacionesDF)

indicativo,latitud,longitud,nombre
X4,41.3839,2.16775,Barcelona
YG,42.51881,1.24244,Tírvia
VY,41.25095,1.29863,Nulles
YD,41.51135,0.85617,Les Borges Blanques
VD,41.68939,1.20381,Els Plans de Sió
WX,41.9178,0.88175,Camarasa
XV,41.48311,2.07956,Sant Cugat del Vallès
U1,42.30648,2.95481,Cabanes
CY,41.87813,2.17873,Muntanyola
D6,42.43515,3.16622,Portbou


In [0]:
geolocator = Nominatim(user_agent="https://maps.googleapis.com")
def getCoordinatesbyMuni(munic):
    """
    Funcion para obtener posicion lat lon dado un municipio
    """
    try:
        location = geolocator.geocode(munic)
    except Exception as e:
        print(e)
        return None
    return f"{location.raw.get('lat')} {location.raw.get('lon')}"

In [0]:
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm

from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="https://maps.googleapis.com")

missing_muniPD = missing_muni.toPandas()

# Se limpia el campo municipio para una mejor precision de la api de google
missing_muniPD['municipio_clean']=missing_muniPD['municipio'].apply(lambda x: x.split(",")[0])
missing_muniPD['municipio_clean'] = missing_muniPD['municipio_clean']+ ', España'

tqdm.pandas()
missing_muniPD['location_code'] = missing_muniPD['municipio_clean'].progress_apply(getCoordinatesbyMuni)

0%| | 0/946 [00:00<?, ?it/s] 0%| | 2/946 [00:00<02:15, 6.99it/s] 0%| | 3/946 [00:00<04:40, 3.36it/s] 0%| | 4/946 [00:01<05:39, 2.78it/s] 1%| | 5/946 [00:01<06:30, 2.41it/s] 1%| | 6/946 [00:02<06:58, 2.25it/s] 1%| | 7/946 [00:02<07:09, 2.18it/s] 1%| | 8/946 [00:03<07:25, 2.11it/s] 1%| | 9/946 [00:03<07:29, 2.08it/s] 1%| | 10/946 [00:04<09:16, 1.68it/s] 1%| | 11/946 [00:04<07:18, 2.13it/s] 1%|▏ | 12/946 [00:05<07:20, 2.12it/s] 1%|▏ | 13/946 [00:05<07:27, 2.08it/s] 1%|▏ | 14/946 [00:06<07:31, 2.06it/s] 2%|▏ | 15/946 [00:06<08:24, 1.85it/s] 2%|▏ | 16/946 [00:07<07:28, 2.07it/s] 2%|▏ | 17/946 [00:07<07:34, 2.04it/s] 2%|▏ | 18/946 [00:08<07:28, 2.07it/s] 2%|▏ | 19/946 [00:08<07:35, 2.03it/s] 2%|▏ | 20/946 [00:09<07:37, 2.02it/s] 2%|▏ | 21/946 [00:09<07:39, 2.02it/s] 2%|▏ | 22/946 [00:10<07:40, 2.01it/s] 2%|▏ | 23/946 [00:10<07:41, 2.00it/s] 3%|▎ | 24/946 [00:11<09:12, 1.67it/s] 3%|▎ | 25/946 [00:11<07:09, 2.14it/s] 3%|▎ | 26/946 [00:12<07:33, 2.03it/s] 3%|▎ | 27/946 [00:13<08:28, 1.81it/s] 3%|▎ | 28/946 [00:13<07:06, 2.15it/s] 3%|▎ | 29/946 [00:13<07:15, 2.11it/s] 3%|▎ | 30/946 [00:14<07:15, 2.10it/s] 3%|▎ | 31/946 [00:14<07:25, 2.05it/s] 3%|▎ | 32/946 [00:15<07:24, 2.06it/s] 3%|▎ | 33/946 [00:15<07:32, 2.02it/s] 4%|▎ | 34/946 [00:16<07:34, 2.01it/s] 4%|▎ | 35/946 [00:16<08:15, 1.84it/s] 4%|▍ | 36/946 [00:17<07:28, 2.03it/s] 4%|▍ | 37/946 [00:17<07:35, 2.00it/s] 4%|▍ | 38/946 [00:18<07:23, 2.05it/s] 4%|▍ | 39/946 [00:18<07:24, 2.04it/s] 4%|▍ | 40/946 [00:19<07:26, 2.03it/s] 4%|▍ | 41/946 [00:19<07:33, 1.99it/s] 4%|▍ | 42/946 [00:20<07:29, 2.01it/s] 5%|▍ | 43/946 [00:20<07:26, 2.02it/s] 5%|▍ | 44/946 [00:21<07:28, 2.01it/s] 5%|▍ | 45/946 [00:21<07:42, 1.95it/s] 5%|▍ | 46/946 [00:22<07:27, 2.01it/s] 5%|▍ | 47/946 [00:22<07:23, 2.03it/s] 5%|▌ | 48/946 [00:23<07:21, 2.03it/s] 5%|▌ | 49/946 [00:23<07:27, 2.00it/s] 5%|▌ | 50/946 [00:24<07:31, 1.98it/s] 5%|▌ | 51/946 [00:24<07:26, 2.00it/s] 5%|▌ | 52/946 [00:25<07:22, 2.02it/s] 6%|▌ | 53/946 [00:25<07:23, 2.01it/s] 6%|▌ | 54/946 [00:26<07:25, 2.00it/s] 6%|▌ | 55/946 [00:26<07:24, 2.00it/s] 6%|▌ | 56/946 [00:27<07:23, 2.01it/s] 6%|▌ | 57/946 [00:27<07:27, 1.99it/s] 6%|▌ | 58/946 [00:28<07:19, 2.02it/s] 6%|▌ | 59/946 [00:28<07:26, 1.99it/s] 6%|▋ | 60/946 [00:29<07:26, 1.99it/s] 6%|▋ | 61/946 [00:29<07:20, 2.01it/s] 7%|▋ | 62/946 [00:30<07:22, 2.00it/s] 7%|▋ | 63/946 [00:30<07:20, 2.01it/s] 7%|▋ | 64/946 [00:31<07:20, 2.00it/s] 7%|▋ | 65/946 [00:31<07:22, 1.99it/s] 7%|▋ | 66/946 [00:32<07:19, 2.00it/s] 7%|▋ | 67/946 [00:32<07:20, 2.00it/s] 7%|▋ | 68/946 [00:33<07:27, 1.96it/s] 7%|▋ | 69/946 [00:33<07:25, 1.97it/s] 7%|▋ | 70/946 [00:34<07:14, 2.02it/s] 8%|▊ | 71/946 [00:34<07:11, 2.03it/s] 8%|▊ | 72/946 [00:35<07:20, 1.98it/s] 8%|▊ | 73/946 [00:35<07:11, 2.02it/s] 8%|▊ | 74/946 [00:36<07:15, 2.00it/s] 8%|▊ | 75/946 [00:36<07:12, 2.01it/s] 8%|▊ | 76/946 [00:37<07:11, 2.02it/s] 8%|▊ | 77/946 [00:37<07:16, 1.99it/s] 8%|▊ | 78/946 [00:38<07:17, 1.99it/s] 8%|▊ | 79/946 [00:38<07:14, 2.00it/s] 8%|▊ | 80/946 [00:39<07:14, 1.99it/s] 9%|▊ | 81/946 [00:40<08:22, 1.72it/s] 9%|▊ | 82/946 [00:40<07:02, 2.05it/s] 9%|▉ | 83/946 [00:40<06:52, 2.09it/s] 9%|▉ | 84/946 [00:41<06:58, 2.06it/s] 9%|▉ | 85/946 [00:41<07:03, 2.03it/s] 9%|▉ | 86/946 [00:42<06:58, 2.05it/s] 9%|▉ | 87/946 [00:42<07:05, 2.02it/s] 9%|▉ | 88/946 [00:43<07:04, 2.02it/s] 9%|▉ | 89/946 [00:43<07:08, 2.00it/s] 10%|▉ | 90/946 [00:44<07:20, 1.94it/s] 10%|▉ | 91/946 [00:44<07:03, 2.02it/s] 10%|▉ | 92/946 [00:45<07:01, 2.03it/s] 10%|▉ | 93/946 [00:45<07:03, 2.01it/s] 10%|▉ | 94/946 [00:46<07:07, 1.99it/s] 10%|█ | 95/946 [00:46<07:00, 2.02it/s] 10%|█ | 96/946 [00:47<07:07, 1.99it/s] 10%|█ | 97/946 [00:47<06:59, 2.02it/s] 10%|█ | 98/946 [00:48<07:02, 2.01it/s] 10%|█ | 99/946 [00:48<07:06, 1.99it/s] 11%|█ | 100/946 [00:49<07:02, 2.00it/s] 11%|█ | 101/946 [00:49<07:02, 2.00it/s] 11%|█ | 102/946 [00:50<07:02, 2.00it/s] 11%|█ | 103/946 [00:51<09:06, 1.5

In [0]:
missing_muniPD[missing_muniPD['location_code'].isna()]

Out[16]:

,codigo_municipio,municipio,municipio_clean,location_code


In [0]:
#Algunos municipios no se han  podido obtener. Se mapean manualmente
missing_muniPD.loc[missing_muniPD['municipio']=='Port de la Selva, el','location_code']='42.3373159 3.2047232'
# missing_muniPD.loc[missing_muniPD['municipio']=='Pont de Bar, el','location_code'] ='42.3721732 1.6047661'
missing_muniPD.loc[missing_muniPD['municipio']=="Vall d'en Bas, la",'location_code'] ='42.1388600 2.4410500'
# missing_muniPD.loc[missing_muniPD['municipio']=="Ametlla de Mar, l'",'location_code'] ='40.8838957 0.8024645'

In [0]:
missing_muniDF=spark.createDataFrame(missing_muniPD).select(["municipio","location_code","codigo_municipio"])

missing_muniDF= missing_muniDF.withColumn("municipio_lat", F.split("location_code"," ").getItem(0)) \
.withColumn("municipio_lon", F.split("location_code"," ").getItem(1))
display(missing_muniDF)

municipio,location_code,codigo_municipio,municipio_lat,municipio_lon
Rupit i Pruit,42.024334 2.4653919,89019,42.024334,2.4653919
Vila-seca,41.1109268 1.144961,431711,41.1109268,1.144961
Alt Àneu,42.69035515 1.0624832582072457,250241,42.69035515,1.0624832582072457
Sant Esteve de Palautordera,41.7040768 2.4343426,82074,41.7040768,2.4343426
"Estany, l'",41.8858007 0.3685293,80790,41.8858007,0.3685293
"Cabanyes, les",39.2908201 -0.5698989,80272,39.2908201,-0.5698989
Sant Martí Sarroca,41.3854214 1.6108158,82270,41.3854214,1.6108158
Siurana,42.2094291 3.0061564,170524,42.2094291,3.0061564
"Nou de Gaià, la",41.1822282 1.3738261,430976,41.1822282,1.3738261
Malla,41.887016 2.2354564,81115,41.887016,2.2354564


In [0]:
# Se ejecuta una crossjoin para tener todas la posibilidades de estaciones y
#municipios con el fin de calcular la distancia entre todos ellos y finalmente obtener la estacion más cercana

munic_estaciones_convinations = missing_muniDF.crossJoin(estacionesDF)
munic_estaciones_convinations.count()

Out[18]: 198660

In [0]:
from geopy.distance import geodesic
from pyspark.sql.types import FloatType

@F.udf(returnType=FloatType())
def geodesic_udf(a, b):
    """
    Funcion para calcular la distancia entre dos puntos de coordinadas
    """
    return geodesic(a, b).m

estacion_munic_distancia = munic_estaciones_convinations.withColumn('estacion_muni_distancia_m', geodesic_udf(F.array("municipio_lat", "municipio_lon"), F.array("latitud", "longitud")))
display(estacion_munic_distancia)

municipio,location_code,codigo_municipio,municipio_lat,municipio_lon,indicativo,latitud,longitud,nombre,estacion_muni_distancia_m
Rupit i Pruit,42.024334 2.4653919,89019,42.024334,2.4653919,X4,41.3839,2.16775,Barcelona,75321.96
Rupit i Pruit,42.024334 2.4653919,89019,42.024334,2.4653919,YG,42.51881,1.24244,Tírvia,114870.95
Rupit i Pruit,42.024334 2.4653919,89019,42.024334,2.4653919,VY,41.25095,1.29863,Nulles,129723.266
Rupit i Pruit,42.024334 2.4653919,89019,42.024334,2.4653919,YD,41.51135,0.85617,Les Borges Blanques,145431.02
Rupit i Pruit,42.024334 2.4653919,89019,42.024334,2.4653919,VD,41.68939,1.20381,Els Plans de Sió,111165.33
Rupit i Pruit,42.024334 2.4653919,89019,42.024334,2.4653919,WX,41.9178,0.88175,Camarasa,131795.7
Rupit i Pruit,42.024334 2.4653919,89019,42.024334,2.4653919,XV,41.48311,2.07956,Sant Cugat del Vallès,68141.54
Rupit i Pruit,42.024334 2.4653919,89019,42.024334,2.4653919,U1,42.30648,2.95481,Cabanes,51164.848
Rupit i Pruit,42.024334 2.4653919,89019,42.024334,2.4653919,CY,41.87813,2.17873,Muntanyola,28786.14
Rupit i Pruit,42.024334 2.4653919,89019,42.024334,2.4653919,D6,42.43515,3.16622,Portbou,73684.72


In [0]:
#Calculamos un ranking de estaciones mas cercanas a mas lejanas
# La estacion con row_number igual a 1 sera la mas cercana
windowSpec  = Window.partitionBy("municipio").orderBy("estacion_muni_distancia_m")

estacion_munic_distancia=estacion_munic_distancia.withColumn("row_number",row_number().over(windowSpec))
display(estacion_munic_distancia)

municipio,location_code,codigo_municipio,municipio_lat,municipio_lon,indicativo,latitud,longitud,nombre,estacion_muni_distancia_m,row_number
Abrera,41.5204464 1.9024126,80018,41.5204464,1.9024126,XC,41.47892,1.97546,Castellbisbal,7646.6675,1
Abrera,41.5204464 1.9024126,80018,41.5204464,1.9024126,CE,41.53109,1.80813,Els Hostalets de Pierola,7957.4097,2
Abrera,41.5204464 1.9024126,80018,41.5204464,1.9024126,D2,41.59252,1.915,Vacarisses,8073.4136,3
Abrera,41.5204464 1.9024126,80018,41.5204464,1.9024126,YK,41.55361,1.99005,Terrassa,8188.393,4
Abrera,41.5204464 1.9024126,80018,41.5204464,1.9024126,WN,41.59539,1.83751,Monistrol de Montserrat,9929.583,5
Abrera,41.5204464 1.9024126,80018,41.5204464,1.9024126,VU,41.63286,1.91718,Rellinars,12545.819,6
Abrera,41.5204464 1.9024126,80018,41.5204464,1.9024126,WY,41.43386,1.79429,Sant Sadurní d'Anoia,13192.308,7
Abrera,41.5204464 1.9024126,80018,41.5204464,1.9024126,XF,41.56568,2.06952,Sabadell,14821.011,8
Abrera,41.5204464 1.9024126,80018,41.5204464,1.9024126,XV,41.48311,2.07956,Sant Cugat del Vallès,15361.012,9
Abrera,41.5204464 1.9024126,80018,41.5204464,1.9024126,D3,41.38197,1.93564,Vallirana,15628.157,10


In [0]:
estacion_munic_distancia= estacion_munic_distancia.filter(F.col("row_number")=="1").select(["codigo_municipio","municipio","indicativo","nombre","row_number"]).withColumnRenamed("nombre","nombre_estacion")

In [0]:
display(estacion_munic_distancia)

codigo_municipio,municipio,indicativo,nombre_estacion,row_number
80018,Abrera,XC,Castellbisbal,1
250045,"Alamús, els",XM,Els Alamús,1
170046,Albons,UB,La Tallada d'Empordà,1
430081,Alfara de Carles,U7,Aldover,1
430094,Alforja,MR,Cornudella de Montsant,1
250143,Alfés,X7,Torres de Segre,1
250169,Alguaire,X3,Alguaire,1
250215,Almenar,WK,Alfarràs,1
250241,Alt Àneu,Z1,Alt Àneu,1
430136,"Ametlla de Mar, l'",UA,L'Ametlla de Mar,1


In [0]:
estacion_munic_distancia.write.mode("overwrite").parquet(f"/mnt/IncendiosForestalesCAT/prep/meteocat/estaciones_cercanas_municipio/")